In [1]:
from __future__ import absolute_import, division, print_function

import os
import sys
import argparse
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from utils import *
from data_utils import AmazonDataset, AmazonDataLoader
from transe_model import KnowledgeEmbedding


logger = None




In [2]:
def train(args):
    dataset = load_dataset(args.dataset)
    print(dataset)
    dataloader = AmazonDataLoader(dataset, args.batch_size)
    words_to_train = args.epochs * dataset.review.word_count + 1

    model = KnowledgeEmbedding(dataset, args).to(args.device)
    logger.info('Parameters:' + str([i[0] for i in model.named_parameters()]))
    
    optimizer = optim.SGD(model.parameters(), lr=args.lr)#随机梯度下降
    steps = 0
    smooth_loss = 0.0

    for epoch in range(1, args.epochs + 1):
        dataloader.reset()
        while dataloader.has_next():
            # Set learning rate.
            lr = args.lr * max(1e-4, 1.0 - dataloader.finished_word_num / float(words_to_train))
            for pg in optimizer.param_groups:
                pg['lr'] = lr

            # Get training batch.
            batch_idxs = dataloader.get_batch()
            batch_idxs = torch.LongTensor(batch_idxs)
            print(batch_idxs)
            
            if len(batch_idxs)<64:
                continue
            #batch_idxs = torch.from_numpy(batch_idxs).to(args.device)

            # Train model.
            optimizer.zero_grad()
            
            train_loss = model(batch_idxs)
            train_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
            optimizer.step()
            smooth_loss += train_loss.item() / args.steps_per_checkpoint

            steps += 1
            
            if steps % args.steps_per_checkpoint == 0:
                logger.info('Epoch: {:02d} | '.format(epoch) +
                            'Words: {:d}/{:d} | '.format(dataloader.finished_word_num, words_to_train) +
                            'Lr: {:.5f} | '.format(lr) +
                            'Smooth loss: {:.5f}'.format(smooth_loss))
                smooth_loss = 0.0

        torch.save(model.state_dict(), '{}/transe_model_sd_epoch_{}.ckpt'.format(args.log_dir, epoch))




In [3]:
def extract_embeddings(args):
    """Note that last entity embedding is of size [vocab_size+1, d]."""
    model_file = '{}/transe_model_sd_epoch_{}.ckpt'.format(args.log_dir, args.epochs)
    print('Load embeddings', model_file)
    state_dict = torch.load(model_file, map_location=lambda storage, loc: storage)
    embeds = {
        USER: state_dict['user.weight'].cpu().data.numpy()[:-1],  # Must remove last dummy 'user' with 0 embed.
        PRODUCT: state_dict['product.weight'].cpu().data.numpy()[:-1],
        WORD: state_dict['word.weight'].cpu().data.numpy()[:-1],
        BRAND: state_dict['brand.weight'].cpu().data.numpy()[:-1],
        CATEGORY: state_dict['category.weight'].cpu().data.numpy()[:-1],
        RPRODUCT: state_dict['related_product.weight'].cpu().data.numpy()[:-1],

        PURCHASE: (
            state_dict['purchase'].cpu().data.numpy()[0],
            state_dict['purchase_bias.weight'].cpu().data.numpy()
        ),
        MENTION: (
            state_dict['mentions'].cpu().data.numpy()[0],
            state_dict['mentions_bias.weight'].cpu().data.numpy()
        ),
        DESCRIBED_AS: (
            state_dict['describe_as'].cpu().data.numpy()[0],
            state_dict['describe_as_bias.weight'].cpu().data.numpy()
        ),
        PRODUCED_BY: (
            state_dict['produced_by'].cpu().data.numpy()[0],
            state_dict['produced_by_bias.weight'].cpu().data.numpy()
        ),
        BELONG_TO: (
            state_dict['belongs_to'].cpu().data.numpy()[0],
            state_dict['belongs_to_bias.weight'].cpu().data.numpy()
        ),
        ALSO_BOUGHT: (
            state_dict['also_bought'].cpu().data.numpy()[0],
            state_dict['also_bought_bias.weight'].cpu().data.numpy()
        ),
        ALSO_VIEWED: (
            state_dict['also_viewed'].cpu().data.numpy()[0],
            state_dict['also_viewed_bias.weight'].cpu().data.numpy()
        ),
        BOUGHT_TOGETHER: (
            state_dict['bought_together'].cpu().data.numpy()[0],
            state_dict['bought_together_bias.weight'].cpu().data.numpy()
        ),
    }
    print(embeds[USER])
    save_embed(args.dataset, embeds)
    return embeds




In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default=BEAUTY, help='One of {beauty, cd, cell, clothing}.')
parser.add_argument('--name', type=str, default='train_transe_model', help='model name.')
parser.add_argument('--seed', type=int, default=123, help='random seed.')
parser.add_argument('--gpu', type=str, default='1', help='gpu device.')
parser.add_argument('--epochs', type=int, default=60, help='number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=64, help='batch size.')
parser.add_argument('--lr', type=float, default=0.5, help='learning rate.')
parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for adam.')
parser.add_argument('--l2_lambda', type=float, default=0, help='l2 lambda')
parser.add_argument('--max_grad_norm', type=float, default=5.0, help='Clipping gradient.')
parser.add_argument('--embed_size', type=int, default=100, help='knowledge embedding size.')
parser.add_argument('--num_neg_samples', type=int, default=5, help='number of negative samples.')
parser.add_argument('--steps_per_checkpoint', type=int, default=200, help='Number of steps for checkpoint.')

_StoreAction(option_strings=['--steps_per_checkpoint'], dest='steps_per_checkpoint', nargs=None, const=None, default=200, type=<class 'int'>, choices=None, help='Number of steps for checkpoint.', metavar=None)

# CLOTH

In [6]:
args = parser.parse_args(['--dataset',CLOTH])
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
if not os.path.isdir(args.log_dir):
    os.makedirs(args.log_dir)

global logger
logger = get_logger(args.log_dir + '/train_log_cloth.txt')
logger.info(args)

set_random_seed(args.seed)

train(args)#

[INFO]  Namespace(batch_size=64, dataset='cloth', device='cpu', embed_size=100, epochs=60, gpu='1', l2_lambda=0, log_dir='./tmp/Amazon_Clothing/train_transe_model', lr=0.5, max_grad_norm=5.0, name='train_transe_model', num_neg_samples=5, seed=123, steps_per_checkpoint=200, weight_decay=0)
dataset_file: ./tmp/Amazon_Clothing/dataset.pkl
[INFO]  Parameters:['purchase', 'mentions', 'describe_as', 'produced_by', 'belongs_to', 'also_bought', 'also_viewed', 'bought_together', 'user.weight', 'product.weight', 'word.weight', 'related_product.weight', 'brand.weight', 'category.weight', 'purchase_bias.weight', 'mentions_bias.weight', 'describe_as_bias.weight', 'produced_by_bias.weight', 'belongs_to_bias.weight', 'also_bought_bias.weight', 'also_viewed_bias.weight', 'bought_together_bias.weight']
[INFO]  Epoch: 04 | Words: 620302/11666341 | Lr: 0.47354 | Smooth loss: 22.93748
[INFO]  Epoch: 07 | Words: 1241025/11666341 | Lr: 0.44696 | Smooth loss: 21.14189
[INFO]  Epoch: 10 | Words: 1867689/11666

# BEAUTY 

In [5]:
args = parser.parse_args(['--dataset',BEAUTY])
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
if not os.path.isdir(args.log_dir):
    os.makedirs(args.log_dir)

global logger
logger = get_logger(args.log_dir + '/train_log_beauty.txt')
logger.info(args)

set_random_seed(args.seed)
train(args)

[INFO]  Namespace(batch_size=64, dataset='beauty', device='cpu', embed_size=100, epochs=60, gpu='1', l2_lambda=0, log_dir='./tmp/Amazon_Beauty/train_transe_model', lr=0.5, max_grad_norm=5.0, name='train_transe_model', num_neg_samples=5, seed=123, steps_per_checkpoint=200, weight_decay=0)
dataset_file: ./tmp/Amazon_Beauty/dataset.pkl
[INFO]  Parameters:['purchase', 'mentions', 'describe_as', 'produced_by', 'belongs_to', 'also_bought', 'also_viewed', 'bought_together', 'user.weight', 'product.weight', 'word.weight', 'related_product.weight', 'brand.weight', 'category.weight', 'purchase_bias.weight', 'mentions_bias.weight', 'describe_as_bias.weight', 'produced_by_bias.weight', 'belongs_to_bias.weight', 'also_bought_bias.weight', 'also_viewed_bias.weight', 'bought_together_bias.weight']
tensor([[ 15924,    874,      4,     -1,     21,  36303,     -1,     -1],
        [  5992,   5382,      5,   1603,      0,    209, 139121,   5886],
        [ 18979,   2518,      1,   1708,     15,  84961,  

RuntimeError: index out of range at c:\n\pytorch_1559129895673\work\aten\src\th\generic/THTensorEvenMoreMath.cpp:191

In [7]:
#args = parser.parse_args(['--dataset',BEAUTY])
train(args)

dataset_file: ./tmp/Amazon_Beauty/dataset.pkl
[INFO]  Parameters:['purchase', 'mentions', 'describe_as', 'produced_by', 'belongs_to', 'also_bought', 'also_viewed', 'bought_together', 'user.weight', 'product.weight', 'word.weight', 'related_product.weight', 'brand.weight', 'category.weight', 'purchase_bias.weight', 'mentions_bias.weight', 'describe_as_bias.weight', 'produced_by_bias.weight', 'belongs_to_bias.weight', 'also_bought_bias.weight', 'also_viewed_bias.weight', 'bought_together_bias.weight']


RuntimeError: index out of range at c:\n\pytorch_1559129895673\work\aten\src\th\generic/THTensorEvenMoreMath.cpp:191

# CD

In [5]:
args = parser.parse_args(['--dataset',CD])
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
if not os.path.isdir(args.log_dir):
    os.makedirs(args.log_dir)

global logger
logger = get_logger(args.log_dir + '/train_log_beauty.txt')
logger.info(args)

set_random_seed(args.seed)

train(args)#CD

[INFO]  Namespace(batch_size=64, dataset='cd', device='cpu', embed_size=100, epochs=60, gpu='1', l2_lambda=0, log_dir='./tmp/Amazon_CDs/train_transe_model', lr=0.5, max_grad_norm=5.0, name='train_transe_model', num_neg_samples=5, seed=123, steps_per_checkpoint=200, weight_decay=0)
dataset_file: ./tmp/Amazon_CDs/dataset.pkl
[INFO]  Parameters:['purchase', 'mentions', 'describe_as', 'produced_by', 'belongs_to', 'also_bought', 'also_viewed', 'bought_together', 'user.weight', 'product.weight', 'word.weight', 'related_product.weight', 'brand.weight', 'category.weight', 'purchase_bias.weight', 'mentions_bias.weight', 'describe_as_bias.weight', 'produced_by_bias.weight', 'belongs_to_bias.weight', 'also_bought_bias.weight', 'also_viewed_bias.weight', 'bought_together_bias.weight']
[INFO]  Epoch: 04 | Words: 601408/11666341 | Lr: 0.47435 | Smooth loss: 22.00827
[INFO]  Epoch: 07 | Words: 1207115/11666341 | Lr: 0.44840 | Smooth loss: 20.70964
[INFO]  Epoch: 10 | Words: 1822605/11666341 | Lr: 0.4

# CELL

In [6]:
train(args) #CELL

dataset_file: ./tmp/Amazon_Cellphones/dataset.pkl
[INFO]  Parameters:['purchase', 'mentions', 'describe_as', 'produced_by', 'belongs_to', 'also_bought', 'also_viewed', 'bought_together', 'user.weight', 'product.weight', 'word.weight', 'related_product.weight', 'brand.weight', 'category.weight', 'purchase_bias.weight', 'mentions_bias.weight', 'describe_as_bias.weight', 'produced_by_bias.weight', 'belongs_to_bias.weight', 'also_bought_bias.weight', 'also_viewed_bias.weight', 'bought_together_bias.weight']
[INFO]  Epoch: 04 | Words: 610917/11666341 | Lr: 0.47395 | Smooth loss: 22.02442
[INFO]  Epoch: 07 | Words: 1228920/11666341 | Lr: 0.44747 | Smooth loss: 20.38819
[INFO]  Epoch: 10 | Words: 1845219/11666341 | Lr: 0.42105 | Smooth loss: 19.53596
[INFO]  Epoch: 13 | Words: 2454394/11666341 | Lr: 0.39492 | Smooth loss: 19.09178
[INFO]  Epoch: 16 | Words: 3057775/11666341 | Lr: 0.36906 | Smooth loss: 18.43481
[INFO]  Epoch: 19 | Words: 3670539/11666341 | Lr: 0.34284 | Smooth loss: 18.23842


# CLOTH

In [5]:
args = parser.parse_args(['--dataset',CLOTH])
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
if not os.path.isdir(args.log_dir):
    os.makedirs(args.log_dir)

global logger
logger = get_logger(args.log_dir + '/train_log_cloth.txt')
logger.info(args)

set_random_seed(args.seed)

train(args)#

[INFO]  Namespace(batch_size=64, dataset='cloth', device='cpu', embed_size=100, epochs=60, gpu='1', l2_lambda=0, log_dir='./tmp/Amazon_Clothing/train_transe_model', lr=0.5, max_grad_norm=5.0, name='train_transe_model', num_neg_samples=5, seed=123, steps_per_checkpoint=200, weight_decay=0)
dataset_file: ./tmp/Amazon_Clothing/dataset.pkl
[INFO]  Parameters:['purchase', 'mentions', 'describe_as', 'produced_by', 'belongs_to', 'also_bought', 'also_viewed', 'bought_together', 'user.weight', 'product.weight', 'word.weight', 'related_product.weight', 'brand.weight', 'category.weight', 'purchase_bias.weight', 'mentions_bias.weight', 'describe_as_bias.weight', 'produced_by_bias.weight', 'belongs_to_bias.weight', 'also_bought_bias.weight', 'also_viewed_bias.weight', 'bought_together_bias.weight']


ValueError: not enough values to unpack (expected 6, got 3)

In [5]:
args = parser.parse_args(['--dataset',CLOTH])
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
if not os.path.isdir(args.log_dir):
    os.makedirs(args.log_dir)

global logger
logger = get_logger(args.log_dir + '/train_log_cloth.txt')
logger.info(args)

set_random_seed(args.seed)

train(args)#

[INFO]  Namespace(batch_size=64, dataset='cloth', device='cpu', embed_size=100, epochs=60, gpu='1', l2_lambda=0, log_dir='./tmp/Amazon_Clothing/train_transe_model', lr=0.5, max_grad_norm=5.0, name='train_transe_model', num_neg_samples=5, seed=123, steps_per_checkpoint=200, weight_decay=0)
dataset_file: ./tmp/Amazon_Clothing/dataset.pkl
[INFO]  Parameters:['purchase', 'mentions', 'describe_as', 'produced_by', 'belongs_to', 'also_bought', 'also_viewed', 'bought_together', 'user.weight', 'product.weight', 'word.weight', 'related_product.weight', 'brand.weight', 'category.weight', 'purchase_bias.weight', 'mentions_bias.weight', 'describe_as_bias.weight', 'produced_by_bias.weight', 'belongs_to_bias.weight', 'also_bought_bias.weight', 'also_viewed_bias.weight', 'bought_together_bias.weight']
[INFO]  Epoch: 04 | Words: 611675/11666341 | Lr: 0.47389 | Smooth loss: 22.93162
[INFO]  Epoch: 07 | Words: 1221355/11666341 | Lr: 0.44780 | Smooth loss: 21.14529
[INFO]  Epoch: 10 | Words: 1839284/11666

epoch: 40
tensor([[ 26125,   1222,      5,     -1,      0, 265394,  80568,  22729],
        [  2878,   6354,      5,    511,     26,  64444,  55725, 167930],
        [  6897,   6612,      1,     -1,     25,  19190,  22912,     -1],
        [  8307,   4876,      1,     -1,    203,  24516, 164804,  91121],
        [ 16479,   3819,      5,     -1,     63,  82904,  70417,     -1],
        [ 14986,   6184,      4,    259,    443, 106767,  43299,     -1],
        [ 15874,   5401,      4,     -1,    521,  34025,  31813,     -1],
        [ 19187,   2435,      4,     -1,     44,  38153,  34589,     -1],
        [  6662,   3211,      5,     -1,     48,  39902,  39697,  14607],
        [ 11560,   1092,      5,     -1,    137,  48450, 100243,  48448],
        [  9481,  10128,      3,     -1,     25, 187505, 200542, 106163],
        [  7668,   3540,      5,     37,      0,  45923,  54867,     -1],
        [ 25465,   5411,      2,    415,     28,  20812, 124015,   4698],
        [  2058,   6901,    

epoch: 40
tensor([[ 20790,   2668,      4,     -1,     97, 257949, 118486, 118486],
        [  4515,   7680,      5,    993,      0, 208930, 146464, 146464],
        [ 23035,   9134,      2,     -1,      0,  34807,     -1,     -1],
        [ 10932,   1626,      3,     -1,    125, 199538,  70499,  70491],
        [ 22700,   6568,      5,     -1,     48, 303781, 261347,     -1],
        [ 25278,   2236,      5,     -1,    788, 202217, 206445,     -1],
        [ 15010,   5169,      5,     -1,      5,  39637,  18713,  18713],
        [ 26467,   1750,      1,    621,    109,  73433,  72782,     -1],
        [ 24715,   5262,      2,    614,     66, 204299, 108750,     -1],
        [ 19875,   6126,      2,     -1,      0, 216220, 194872, 194872],
        [ 19750,   3413,      1,     -1,    267,  38726,  47272,     -1],
        [  1725,   2172,      3,     -1,    507, 145977,     -1,     -1],
        [ 26867,   6854,      4,     -1,     45,  53582, 204769,     -1],
        [   383,   2197,    

epoch: 40
tensor([[  1724,   2878,      5,     -1,     42,  13772,  20426,     -1],
        [  9460,   8564,      5,     -1,    149, 114218, 114219,  41825],
        [ 19545,   6530,      4,     17,      0,  48165,   1398,     -1],
        [  8137,   5441,      4,     -1,     45, 108000, 107988,     -1],
        [  4899,   5495,      5,     -1,      5,  31897,  31892,     -1],
        [   346,   9494,      5,     -1,    441,  31884,  22854,     -1],
        [  2149,   2634,      5,    746,      2,  39353,   2292,   2322],
        [  7918,   9782,      5,     -1,     69,   6435,  16520,   6435],
        [ 25518,   9834,      1,     -1,     93,   1597,   3819,   3825],
        [ 15176,   3924,      3,     -1,     27,  31513,    923,   9468],
        [ 19973,   3762,      5,     -1,    106,  30140,   6057,  13241],
        [ 26347,   9343,      4,     -1,     97,     -1,     -1,     -1],
        [ 21890,   4421,      4,     -1,    149, 231245, 219055,     -1],
        [ 26068,   4228,    

epoch: 40
tensor([[  8998,    896,      5,     32,    124,   4423,  34455,   9961],
        [ 10685,   4053,      5,     -1,     19,  69241,  69241,  46795],
        [  9101,   8991,      5,     74,     38,  46502,  17022,   8821],
        [ 15011,   3819,      5,     -1,      0, 129820,   7277,     -1],
        [ 21987,   7649,      1,     -1,      2,  21798,  21767,  20984],
        [ 13014,   5983,      5,     -1,     15,  26364,     -1,     -1],
        [ 22538,   2553,      1,     -1,      1,   3701,  53721,     -1],
        [  7424,   5520,      5,     -1,     69, 120605, 153571,  31554],
        [ 18173,   2135,      5,     -1,      1, 102671,  60302,     -1],
        [ 13130,   3068,      5,     -1,     48,  16181,  16277,  16277],
        [ 24000,   4382,      1,     -1,      5,   6855,     -1,   6906],
        [ 14092,   3332,      3,     -1,     51, 145781,  15693,  15707],
        [  5723,   5434,      4,     -1,    346,  16493,     -1,     -1],
        [ 22045,   2864,    

...

epoch: 40
tensor([[ 18939,   3100,      2,     -1,      0, 107288,   4847,   4847],
        [ 24245,   8564,      5,     -1,     19,   7044,  55188,   5021],
        [   153,   3634,      5,     -1,     86,  37729,  61455,     -1],
        [  4208,   1927,      2,     -1,      0,  49095, 233954,     -1],
        [ 16685,   2701,      1,     -1,      0, 198295,  94926,     -1],
        [ 21481,   9013,      1,     -1,     42,  31960,   1720,   1720],
        [ 27332,   9542,      1,     -1,     25,   2442,   2504,     -1],
        [ 10054,   5174,      1,     -1,     19, 228926, 181012,     -1],
        [ 15054,   7119,      2,    162,     28, 121634,  22533,  22533],
        [ 12369,   2560,      5,     -1,    101,   6996,   7002,     -1],
        [ 25551,   1974,      5,     -1,      0, 261067, 128549,     -1],
        [  7452,    592,      5,     -1,     28,  42785,  11096,   5231],
        [ 27083,      0,      2,     -1,      0, 172589, 148899,  40113],
        [ 22814,   8418,    

epoch: 40
tensor([[ 17754,   9862,      3,     -1,     28,   5377,  43768,     -1],
        [ 23858,   6837,      3,     -1,    184,  20580,  20389,     -1],
        [ 19477,   5434,      1,     -1,      0, 135173,     -1,     -1],
        [ 10970,   8811,      5,     -1,     45, 176542, 172702,     -1],
        [  7011,   7313,      5,     -1,     62, 174065,  80464,  83139],
        [  1721,   5637,      4,     -1,    281, 134137,  19933,  19957],
        [ 17105,   5168,      4,     -1,     25,  14741,  43475,     -1],
        [ 16648,   8400,      4,     -1,     19,  93515,  93520,     -1],
        [  8574,   2719,      5,    359,      0,  18097,  27730,     -1],
        [  6668,   7057,      5,     -1,      2,  35714,   2350, 226635],
        [ 18753,   2401,      5,     -1,     25, 172418,  17157,  17134],
        [ 12698,   2554,      2,     -1,      2,  97289,     -1,     -1],
        [ 26989,   9023,      5,     -1,    179,   9551,   9539,     -1],
        [ 19073,   4900,    

epoch: 40
tensor([[  1702,   6396,      3,     -1,    375,  18433, 157827,  46795],
        [  4273,   4801,      5,      9,     39,   1362,  46612,     -1],
        [  2767,   9254,      5,     -1,    267,  18535,  30056,     -1],
        [  8424,   3758,      4,      6,      5,  37181, 122270,     -1],
        [  6087,   9762,      4,     -1,      2, 254428,     -1,     -1],
        [  4593,   3159,      4,     -1,      0,   3163,   3171,   2725],
        [ 10392,   6768,      5,     -1,     25, 102343,  74476,  30482],
        [  7191,     35,      1,     -1,    593, 160270, 103813, 103813],
        [  4559,   2318,      1,     17,    155,   8779,   8863,     -1],
        [  1152,   4530,      4,     -1,     48,  53181,     -1,     -1],
        [ 11539,  10207,      4,     -1,      2,   1543, 127109,     -1],
        [ 16523,   2983,      5,     -1,      2,  55737,   8315,     -1],
        [ 27598,   8192,      5,     -1,     91,  10383,   3620,     -1],
        [  7783,   9858,    

epoch: 40
tensor([[  4997,   1081,      5,    218,     25, 141136,  53121,     -1],
        [ 25688,   6416,      4,     -1,     48, 236325,     -1,     -1],
        [  2844,   9226,      5,     -1,    257,  31301,   9936,     -1],
        [ 19123,   5455,      1,     -1,    101,   2584,   7002,     -1],
        [  3669,   4621,      3,     -1,     25,  45844,  89009,  59250],
        [ 19234,   8535,      1,     -1,    164,  11099,  18505,     -1],
        [ 16113,   2583,      4,     -1,      0, 118464, 292936, 118453],
        [ 17765,   2397,      3,     -1,    413,  17930,   3341,  40098],
        [  1346,   4200,      1,     -1,      5,  22729,  44350,   6468],
        [ 17744,   2421,      5,     -1,      2,  27930,   3825,     -1],
        [  9775,   3635,      3,    329,      0,  72566,  91034,    636],
        [  6803,   1497,      5,     -1,      3,   9449,  13336,  13336],
        [ 26315,   8772,      5,     -1,     46,  38561,  43357,     -1],
        [ 14493,   8572,    

UnboundLocalError: local variable 'data' referenced before assignment

In [9]:
emb = extract_embeddings(args)

Load embeddings ./tmp/Amazon_Cellphones/train_transe_model/transe_model_sd_epoch_60.ckpt
[[-0.00211423  0.00353241 -0.00094879 ...  0.00394515  0.00175596
   0.00451101]
 [ 0.003749    0.00302171  0.00130764 ... -0.00449195  0.00298442
   0.0036598 ]
 [ 0.00138646 -0.0020297   0.00209535 ... -0.00067849 -0.0029328
  -0.00129671]
 ...
 [-0.00076333  0.00042423  0.00172309 ... -0.00453087  0.00054564
   0.00016628]
 [-0.00158372  0.00021306  0.00426845 ... -0.00221793  0.00039889
  -0.0011367 ]
 [-0.0010701  -0.00076093 -0.00179834 ... -0.00145278 -0.0025443
  -0.00503418]]


In [12]:
'''
USER: state_dict['user.weight'].cpu().data.numpy()[:-1],  # Must remove last dummy 'user' with 0 embed.
        PRODUCT: state_dict['product.weight'].cpu().data.numpy()[:-1],
        WORD: state_dict['word.weight'].cpu().data.numpy()[:-1],
        BRAND: state_dict['brand.weight'].cpu().data.numpy()[:-1],
        CATEGORY: state_dict['category.weight'].cpu().data.numpy()[:-1],
        RPRODUCT: state_dict['related_product.weight'].cpu().data.numpy()[:-1],
        
'''

print('user:',emb[USER].shape)
print('prod:',emb[PRODUCT].shape)
print('word:',emb[WORD].shape)
print('brand:',emb[BRAND].shape)
print('related_prod:',emb[RPRODUCT].shape)
print('self_loop:',emb[SELF_LOOP].shape)

user: (27879, 100)
prod: (10429, 100)
word: (22493, 100)
brand: (955, 100)
related_prod: (101287, 100)


KeyError: 'self_loop'

In [19]:
emb[PURCHASE][0].shape

(100,)

In [20]:
emb[PURCHASE][1].shape

(10430, 1)

In [17]:
def load_embed(dataset):
    embed_file = '{}/transe_embed.pkl'.format(TMP_DIR[dataset])
    print('Load embedding:', embed_file)
    embed = pickle.load(open(embed_file, 'rb'))
    return embed
embeding = load_embed(CELL)

In [26]:
len(embeding['user'])
embeding['user'].shape

(27879, 100)

In [13]:
user
product
word
brand
category
realated_product
purchase
mentions
described_as
produced_by
belongs_to
also_bought
also_viewed
bought_together


NoneType

In [ ]:
'''
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str, default=BEAUTY, help='One of {beauty, cd, cell, clothing}.')
    parser.add_argument('--name', type=str, default='train_transe_model', help='model name.')
    parser.add_argument('--seed', type=int, default=123, help='random seed.')
    parser.add_argument('--gpu', type=str, default='1', help='gpu device.')
    parser.add_argument('--epochs', type=int, default=30, help='number of epochs to train.')
    parser.add_argument('--batch_size', type=int, default=64, help='batch size.')
    parser.add_argument('--lr', type=float, default=0.5, help='learning rate.')
    parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for adam.')
    parser.add_argument('--l2_lambda', type=float, default=0, help='l2 lambda')
    parser.add_argument('--max_grad_norm', type=float, default=5.0, help='Clipping gradient.')
    parser.add_argument('--embed_size', type=int, default=100, help='knowledge embedding size.')
    parser.add_argument('--num_neg_samples', type=int, default=5, help='number of negative samples.')
    parser.add_argument('--steps_per_checkpoint', type=int, default=200, help='Number of steps for checkpoint.')
    args = parser.parse_args()

    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

    args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
    if not os.path.isdir(args.log_dir):
        os.makedirs(args.log_dir)

    global logger
    logger = get_logger(args.log_dir + '/train_log.txt')
    logger.info(args)

    set_random_seed(args.seed)
    train(args)
    extract_embeddings(args)


if __name__ == '__main__':
    main()

'''

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default=BEAUTY, help='One of {beauty, cd, cell, clothing}.')
parser.add_argument('--name', type=str, default='train_transe_model', help='model name.')
parser.add_argument('--seed', type=int, default=123, help='random seed.')
parser.add_argument('--gpu', type=str, default='1', help='gpu device.')
parser.add_argument('--epochs', type=int, default=30, help='number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=64, help='batch size.')
parser.add_argument('--lr', type=float, default=0.5, help='learning rate.')
parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for adam.')
parser.add_argument('--l2_lambda', type=float, default=0, help='l2 lambda')
parser.add_argument('--max_grad_norm', type=float, default=5.0, help='Clipping gradient.')
parser.add_argument('--embed_size', type=int, default=100, help='knowledge embedding size.')
parser.add_argument('--num_neg_samples', type=int, default=5, help='number of negative samples.')
parser.add_argument('--steps_per_checkpoint', type=int, default=200, help='Number of steps for checkpoint.')
args = parser.parse_args()

#os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
#args.device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

args.log_dir = '{}/{}'.format(TMP_DIR[args.dataset], args.name)
if not os.path.isdir(args.log_dir):
    os.makedirs(args.log_dir)

global logger
logger = get_logger(args.log_dir + '/train_log.txt')
logger.info(args)

set_random_seed(args.seed)
train(args)
extract_embeddings(args)

In [ ]:
'''
bash
python train_transe_model.py --dataset <dataset_name>
'''

In [6]:
a = [[1,2,3],[]]

In [7]:
a[-1]

[]

In [8]:
len(a[-1])

0